## Lab 4 - Cross-encoder re-ranking

In [18]:
# from helper_utils import load_chroma, word_wrap, project_embeddings
# from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
# import numpy as np

In [19]:
# embedding_function = SentenceTransformerEmbeddingFunction()

# chroma_collection = load_chroma(filename='microsoft_annual_report_2022.pdf', collection_name='microsoft_annual_report_2022', embedding_function=embedding_function)
# chroma_collection.count()

In [20]:
import os
import sys
sys.path.append('../..')
import chromadb
import numpy as np
# import umap.umap_ as umap

from helper_utils_02 import load_pdf_chroma, word_wrap, project_embeddings,flatten
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from py3810.myUtils import pickle_dump, pickle_load

In [21]:
# Specify the directory to store the database files
path_chromadb_dump = '../Chromadb/'
persist_dir = '../Chromadb/collections/'
is_persistent = True

filename_pdf = 'microsoft_annual_report_2022.pdf'
collection_name = 'microsoft_annual_report_2022'
embedding_function = SentenceTransformerEmbeddingFunction()

In [22]:
if is_persistent:
  try:
    client = chromadb.PersistentClient(path=persist_dir)
    chroma_collection = client.get_collection(name=collection_name, embedding_function=embedding_function)
    embeddings = pickle_load(filename_pickle='_embeddings', path_pickle_dump=path_chromadb_dump)
    umap_transform = pickle_load(filename_pickle='_umap_transform', path_pickle_dump=path_chromadb_dump)
    projected_dataset_embeddings = pickle_load(filename_pickle='_projected_dataset_embeddings', path_pickle_dump=path_chromadb_dump)
    print(f'loaded embeddings, umpa_transform, projected_dataset_embeddings')
  except FileNotFoundError:
    chroma_collection = \
      load_pdf_chroma(
        filename_pdf=filename_pdf,
        collection_name=collection_name,
        embedding_function=embedding_function,
        persist_directory=persist_dir,
        is_persistent=is_persistent,
      )
    print(f'Created chroma_collection {collection_name}')
    print(f'chroma_collection.count(): {chroma_collection.count()}')
    embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
    umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
    projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)
    print(f'created embeddings, umpa_transform, projected_dataset_embeddings')
    pickle_dump(file_to_pickle=embeddings, filename_pickle='_embeddings', path_pickle_dump=path_chromadb_dump)
    pickle_dump(file_to_pickle=umap_transform, filename_pickle='_umap_transform', path_pickle_dump=path_chromadb_dump)
    pickle_dump(file_to_pickle=projected_dataset_embeddings, filename_pickle='_projected_dataset_embeddings', path_pickle_dump=path_chromadb_dump)    
    print(f'saved embeddings, umpa_transform, projected_dataset_embeddings')
else:
  if os.path.exists(persist_dir):
    client = chromadb.PersistentClient(path=persist_dir)
    chroma_collection = client.get_collection(name=collection_name, embedding_function=embedding_function)
    print(f'Loaded chroma_collection {collection_name}')
    embeddings = pickle_load(filename_pickle='_embeddings', path_pickle_dump=path_chromadb_dump)
    umap_transform = pickle_load(filename_pickle='_umap_transform', path_pickle_dump=path_chromadb_dump)
    projected_dataset_embeddings = pickle_load(filename_pickle='_projected_dataset_embeddings', path_pickle_dump=path_chromadb_dump)
    print(f'loaded embeddings, umpa_transform, projected_dataset_embeddings')
    print(f'chroma_collection.count(): {chroma_collection.count()}')
  else:
    chroma_collection = \
      load_pdf_chroma(
        filename_pdf=filename_pdf,
        collection_name=collection_name,
        embedding_function=embedding_function,
        persist_directory=persist_dir,
        is_persistent=True,  # Set True to save collection, can always delete directory
      )
    print(f'Created chroma_collection {collection_name}')
    print(f'chroma_collection.count(): {chroma_collection.count()}')
    embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
    umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
    projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)
    print(f'created embeddings, umpa_transform, projected_dataset_embeddings')
    pickle_dump(file_to_pickle=embeddings, filename_pickle='_embeddings', path_pickle_dump=path_chromadb_dump)
    pickle_dump(file_to_pickle=umap_transform, filename_pickle='_umap_transform', path_pickle_dump=path_chromadb_dump)
    pickle_dump(file_to_pickle=projected_dataset_embeddings, filename_pickle='_projected_dataset_embeddings', path_pickle_dump=path_chromadb_dump)    
    print(f'saved embeddings, umpa_transform, projected_dataset_embeddings')        

loaded embeddings, umpa_transform, projected_dataset_embeddings


# Re-ranking the long tail

In [23]:
query = "What has been the investment in research and development?"
results = chroma_collection.query(query_texts=query, n_results=10, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

assessment of key technology trends, we maintain our long - term
commitment to research and development across a wide spectrum of
technologies, tools, and platforms spanning digital work and life
experiences, cloud computing, ai, devices, and operating systems. while
our main product research and development facilities are located in
redmond, washington, we also operate research and development
facilities in other parts of the u. s. and around the world. this
global approach helps us remain competitive in local markets and
enables us to continue to attract top talent from across the world. we
plan to continue to make significant investments in a broad range of
product research and development activities, and as appropriate we will
coordinate our research and development across operating segments and
leverage the results across the company. in addition to our main
research and development operations, we also operate microsoft
research. microsoft research is

and business methods, we bel

In [24]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [25]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print("Scores:")
for score in scores:
    print(score)

Scores:
1.6141684
-0.0895655
-2.7713428
-10.692507
-10.671734
-0.40982634
-11.017633
-1.2966431
-8.666394
-10.632914


In [26]:
print("New Ordering:")
# if the index for the largest number is 5, than the 1st number printed is 5
for o in np.argsort(scores)[::-1]:
    print(o)

New Ordering:
0
1
5
7
2
8
9
4
3
6


# Re-ranking with Query Expansion

In [27]:
original_query = "What were the most important factors that contributed to increases in revenue?"
generated_queries = [
    "What were the major drivers of revenue growth?",
    "Were there any new product launches that contributed to the increase in revenue?",
    "Did any changes in pricing or promotions impact the revenue growth?",
    "What were the key market trends that facilitated the increase in revenue?",
    "Did any acquisitions or partnerships contribute to the revenue growth?"
]

In [36]:
queries = [original_query] + generated_queries

results = chroma_collection.query(query_texts=queries, n_results=10, include=['documents', 'embeddings'])
retrieved_documents = results['documents']

In [40]:
# Deduplicate the retrieved documents
document_count = 0
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        document_count += 1
        unique_documents.add(document)

unique_documents = list(unique_documents)
print(f'processed {document_count} documents, {len(unique_documents)} are unique')

processed 60 documents, 24 are unique


In [30]:
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])

In [31]:
scores = cross_encoder.predict(pairs)

In [44]:
print(f"len(scores): {len(scores)}")
print("Scores:")
for score in scores:
    print(score)

len(scores): 24
Scores:
-10.540945
-3.8934517
-10.047086
-8.948592
-9.798314
-5.497715
-4.6518936
-4.6286144
-6.1867895
-4.808131
-10.0839405
-9.056409
-8.505108
-5.831361
-5.3565664
-11.083161
-7.4323034
-10.148884
-4.7030272
-3.768154
-11.0792675
-4.8184853
-1.5624232
-9.807878


In [33]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o)

New Ordering:
22
19
1
7
6
18
9
21
14
5
13
8
16
12
3
11
4
23
2
10
17
0
20
15


In [ ]:
#TODO map the first 5